In [2]:
import pandas as pd
import pyodbc

In [51]:
username = 'vigrose'
password = 'Hufflepuff10946'
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

order_query= \
    f""" 
    SELECT DISTINCT 
    D.FULL_DT,
    D.YR,
    D.MONTH_NBR,
    H.MED_EDU_NBR AS ME,
    H.PARTY_ID,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND D.YR = 2020
    """

customer_query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c  
    """    

def fix_me(me_list):
    nums = []
    for num in me_list:
        num = str(num)
        num = num.replace('.0', '')
        if len(num) == 10:
            num = '0' + num
        elif len(num) == 9:
            num = '00' + num
        elif len(num) == 8:
            num = '000' + num
        nums.append(num)
    return nums

In [5]:
ppd = pd.read_csv("../../Data/Scorecard/ppd_data_20200704.csv", low_memory=False)
ppd['ME']=fix_me(ppd['ME'])

In [52]:
orders = pd.read_sql(con=AMADM, sql=order_query)
print(len(orders))

352617


In [7]:
# #Execute queries

# customers = pd.read_sql(con=AMADM, sql=customer_query)
# print(len(customers))
# #Filter
# customers.CUSTOMER_KEY = customers.CUSTOMER_KEY.astype(str)
# customers = customers.fillna('None')
class_cust = customers[(customers.CUSTOMER_CATEGORY_DESC !='None')&(customers.CUSTOMER_NAME != 'None')]
#     if order_type == 'reapp':
#         orders = orders[orders.ORDER_PRODUCT_ID.isin([4915522,4915514])]
#     elif order_type == 'app':
#         orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513,4915521,4915502])]
orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513,4915521,4915502,4915522,4915514])]
physician_orders = pd.merge(orders, ppd, on='ME')
print(len(orders))
print(len(class_cust))
results = pd.merge(orders, class_cust, on = 'CUSTOMER_KEY').drop_duplicates()

2350036
22902


In [8]:
len(results)

2336540

In [15]:
results.drop_duplicates('CUSTOMER_NBR')

,FULL_DT,YR,MONTH_NBR,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
0,2014-12-30,2014,12,-1,-1,03221922,4915514,-1,82,000002040899,07HP51677,Children'S Hospital Colorado,Eprofile Webstore Cusomer,Eprofile,Hospital
2100,2015-06-01,2015,6,-1,-1,03271395,4915513,-1,937,000002081394,14MG74300,Hawaii Medical Service Association,Eprofile Webstore Cusomer,Eprofile,Managed Care
2414,2015-06-01,2015,6,-1,-1,03271396,4915513,-1,3062,000002086333,08HP62481,St. Mary'S Hospital,Eprofile Webstore Cusomer,Eprofile,Hospital
2549,2015-06-01,2015,6,-1,-1,03271397,4915513,-1,6798,000002102430,35GO51699,Bath Vamc,Eprofile Webstore Cusomer,Eprofile,Government
2560,2015-06-01,2015,6,-1,-1,03271398,4915513,-1,4037,000002101012,26HR62243,Consulting Radiologists Ltd,Eprofile Webstore Cusomer,Eprofile,Health Related
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336518,2020-06-19,2020,6,01205100811,4514014,43337406,4915513,10702740,137445,000003914333,None,Rsps Fayetteville Asc Lllp,Eprofile Webstore Customer,Eprofile,Ambulatory Care
2336519,2020-07-09,2020,7,04114110138,4827105,43436566,4915521,10722701,138052,000003841423,None,Tova Surgical Center Corporation,Eprofile Webstore Customer,Eprofile,Ambulatory Care
2336525,2020-07-21,2020,7,03509141303,5877973,43461100,4915513,10794416,138462,000003425247,None,Huggins Hospital,Eprofile Webstore Customer,Eprofile,Hospital
2336532,2020-07-06,2020,7,00512151078,6156158,43423103,4915513,11046286,137922,000003313285,None,Erlanger Behavioral Health Hospital,Eprofile Webstore Customer,Eprofile,Hospital


In [25]:
results[results['ORDER_PRODUCT_ID'].isin([4915513,4915521,4915502])].drop_duplicates(['CUSTOMER_NBR','ME','YR']).groupby('YR').count()[['FULL_DT']]

,FULL_DT
YR,
2014,46
2015,71936
2016,356716
2017,369406
2018,353781
2019,367165
2020,218327


In [ ]:
#     if order_type == 'reapp':
#         orders = orders[orders.ORDER_PRODUCT_ID.isin([4915522,4915514])]
#     elif order_type == 'app':
#         orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513,4915521,4915502])]

In [47]:
results

,FULL_DT,YR,MONTH_NBR,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC,NEW,REAPP
0,2014-12-30,2014,12,-1,-1,03221922,4915514,-1,82,000002040899,07HP51677,Children'S Hospital Colorado,Eprofile Webstore Cusomer,Eprofile,Hospital,False,True
1,2016-04-05,2016,4,02878860023,2173743,36905024,4915513,424,82,000002040899,07HP51677,Children'S Hospital Colorado,Eprofile Webstore Cusomer,Eprofile,Hospital,True,False
2,2016-06-02,2016,6,02878860023,2173743,37139280,4915513,424,82,000002040899,07HP51677,Children'S Hospital Colorado,Eprofile Webstore Cusomer,Eprofile,Hospital,True,False
3,2016-07-05,2016,7,02802770807,2170087,37215822,4915513,1763,82,000002040899,07HP51677,Children'S Hospital Colorado,Eprofile Webstore Cusomer,Eprofile,Hospital,True,False
4,2016-06-09,2016,6,02802910021,2170829,37153841,4915513,2140,82,000002040899,07HP51677,Children'S Hospital Colorado,Eprofile Webstore Cusomer,Eprofile,Hospital,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2336535,2020-07-23,2020,7,03508131169,5606406,43464817,4915513,11231350,60131,000002100353,38AM30529,Tri State Endoscopy Center,Eprofile Webstore Customer,Eprofile,Ambulatory Care,True,False
2336536,2020-07-20,2020,7,01103120574,5138135,43458709,4915513,11276866,57024,000002080964,48PP70176,Baylormedcare,Eprofile Webstore Customer,Eprofile,Managed Care,True,False
2336537,2020-07-10,2020,7,01201131185,5603321,43438396,4915513,11286153,52723,000002552171,None,Castle Surgicenter Llc,Eprofile Webstore Customer,Eprofile,Ambulatory Care,True,False
2336538,2020-07-20,2020,7,03006171604,6732225,43458069,4915513,11318994,138390,000002079897,48HR08416,Cypress Healthcare Consultants,Eprofile Webstore Customer,Eprofile,Managed Care,True,False


In [31]:
results = results.fillna('None')

In [50]:
results[results['CUSTOMER_CATEGORY_DESC']=='Hospital'].drop_duplicates(['YR','CUSTOMER_NBR']).groupby(['YR']).count()[['FULL_DT']]

,FULL_DT
YR,
2014,4
2015,3052
2016,3615
2017,3504
2018,3385
2019,3233
2020,2937


In [40]:
results['NEW'] = results['ORDER_PRODUCT_ID'].isin([4915513,4915521,4915502])

In [41]:
results['REAPP'] = results['ORDER_PRODUCT_ID'].isin([4915522,4915514])

In [55]:
results = pd.merge(orders, customers, on = 'CUSTOMER_KEY')

In [57]:
pd.merge(results, ppd, on='ME')

,FULL_DT,YR,MONTH_NBR,ME,PARTY_ID,ORDER_NBR,ORDER_PRODUCT_ID,ORDER_PHYSICIAN_HIST_KEY,CUSTOMER_KEY,CUSTOMER_NBR,...,POLO_STATE,POLO_ZIP,POLO_SECTOR,POLO_CARRIER_ROUTE,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
0,2020-01-02,2020,1,00518870327,1922532,42695399,4915513,7675154,59051,000002085085,...,CA,94550,805.0,C062,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-04-24,2020,4,01205970734,1840235,43222374,4915513,7687513,59051,000002085085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-07-16,2020,7,01205970734,1840235,43452623,4915513,9881172,53259,000002080263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-09,2020,3,01205970734,1840235,43122403,4915513,7687513,62996,000002084259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-06-01,2020,6,00515940189,1898559,43299335,4915513,7689769,59051,000002085085,...,CA,94566,4150.0,C033,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340857,2020-07-24,2020,7,03508030109,2954322,43466402,4915491,11390249,138516,003508030109,...,RI,02903,4144.0,C019,NaN,NaN,NaN,NaN,NaN,NaN
340858,2020-07-24,2020,7,03508030109,2954322,43466402,4915515,11390249,138516,003508030109,...,RI,02903,4144.0,C019,NaN,NaN,NaN,NaN,NaN,NaN
340859,2020-07-24,2020,7,04901090781,3605787,43467462,4915515,11390366,138493,004901090781,...,TX,77054,1940.0,C020,NaN,NaN,NaN,NaN,NaN,NaN
340860,2020-07-24,2020,7,00514120943,5163838,43467735,4915515,11390437,138507,000514120943,...,CA,91342,1437.0,C012,NaN,NaN,NaN,NaN,NaN,NaN
